In [ ]:
#Function to separate categorical,continuous and date features 
def separate_cont_and_cat_features(df):
    
    categorical_features = df.select_dtypes(include=['object','bool','category']).columns.tolist()
    continuous_features  = df.select_dtypes(include=['number']).columns.tolist()
    date_features        = df.select_dtypes(include=['datetime64[ns]']).columns.tolist()
    
    return continuous_features,categorical_features,date_features

In [ ]:
#Function to split timestamp/date column into sub features i.e., 'Year', 'Month',  'Day', 'Hour' ,'Minute','Second'
def date_splitter(df, fldname):
    
    fld = df[fldname] 
       
    targ_pre = re.sub('[Dd]ate$', '', fldname)#substitute the phrase 'date' by spaces in the original field name
    
    #obtain different sub features for the date column
    for n in ('Year', 'Month',  'Day', 'Hour' ,'Minute','Second'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
        #drop 'Hour' ,'Minute','Second' for date features as they will be zeros
        if n in ['Hour' ,'Minute','Second']:
            if any(df[targ_pre+n] > 0):   
                df[targ_pre+n]=df[targ_pre+n].astype('category')#convert new sub features to category format
            else:
                df.drop(targ_pre+n,axis=1,inplace=True)
        else:
            df[targ_pre+n]=df[targ_pre+n].astype('str')    
            df[targ_pre+n]=df[targ_pre+n].astype('category')#convert new sub features to category format

    return df

In [ ]:
#Function to convert date field to its Julian format
def Juliandtconv(df,fldname):
    
    df[fldname] = df[fldname].apply(lambda x: x.to_julian_date())
            
    return  df

In [ ]:

continuous_features,categorical_features,date_features = separate_cont_and_cat_features(data)

if date_features:
    print("Transforming date features in the data...")
    print('\n')
    print("The date features in the dataset are : ", date_features)
    print('\n')
    
    first_dt_feature=True
    
    for i in date_features:
        #if target is a date/timestamp, then user gets the option to convert it to Julian format or retain as it is          
        if i == target_name:
            split_or_julian_ind=''
            print("Please specify whether you would like to convert the target feature,{} to Julian format or retain as it is".format(i))
            while(split_or_julian_ind != 'J' and split_or_julian_ind != 'R'):
                split_or_julian_ind=(input("Enter J to convert to Julian or R to retain existing date format : ")).upper()
            print("\n")
            if split_or_julian_ind == 'J':
                data = Juliandtconv(data,fldname=i)
            elif split_or_julian_ind == 'R':
                pass
        else:
            #for all date features user is given the option to convert it to Julian or split it 
            #all date features can be converted to Julian format/split simultaneously or user can individually specify which date feature is to be split and which is to be converted to Julian 
            if first_dt_feature:
                first_dt_feature = False
                allind=(input("Would you like to split all date feaures or convert all date features to Julian format? - Y/N")).upper()
                if allind == 'Y':
                    split_or_julian_ind=''
                    while(split_or_julian_ind != 'S' and split_or_julian_ind != 'J'):
                        split_or_julian_ind=(input("Enter S to split, J to convert to Julian format : ")).upper()
                        print("\n")
            if allind != 'Y': 
                print("Please specify whether you would like to split {} or convert it to Julian format ".format(i))
                split_or_julian_ind=''
                while(split_or_julian_ind != 'S' and split_or_julian_ind != 'J'):
                    split_or_julian_ind=(input("Enter S to split, J to convert to Julian format : ")).upper()
                    print("\n")
            if split_or_julian_ind == 'S':
                data = date_splitter(data, fldname=i)
                data.drop(i,axis=1,inplace=True)
            elif split_or_julian_ind == 'J':
                data = Juliandtconv(data,fldname=i)

          
    print('Date tarnsformed data:\n')
    display(data.head())
